## Part 1: Preprocessing

In [156]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers


#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [157]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [158]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [159]:
# Create a list of at least 10 column names to use as X data
X = ['Age', 'DistanceFromHome', 'Education', 'JobSatisfaction',
     'NumCompaniesWorked', 'OverTime', 'StockOptionLevel',
     'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion']

# Create X_df using your selected columns
X_df = attrition_df[X]

# Show the data types for X_df
X_df.dtypes

,0
Age,int64
DistanceFromHome,int64
Education,int64
JobSatisfaction,int64
NumCompaniesWorked,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64


In [160]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [161]:
X_train['OverTime'].value_counts()

,count
OverTime,
No,780
Yes,322


In [162]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
def encode_overtime(overtime):
    if overtime == 'Yes':
        return 1.0
    else:
        return 0.0

X_train['OverTime'] = X_train['OverTime'].apply(encode_overtime)
X_test['OverTime'] = X_test['OverTime'].apply(encode_overtime)

X_train.head(10)

,Age,DistanceFromHome,Education,JobSatisfaction,NumCompaniesWorked,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion
1343,29,7,3,1,3,0.0,0,3,3,1
1121,36,1,4,3,6,0.0,0,3,1,0
1048,34,3,3,1,3,0.0,0,3,13,3
1393,27,9,3,4,1,0.0,0,3,7,0
527,32,10,3,4,1,0.0,0,2,10,0
1138,50,20,5,3,2,1.0,1,3,30,12
844,30,10,3,3,1,0.0,1,3,10,1
370,21,12,3,2,1,0.0,0,3,1,0
54,26,23,3,4,7,1.0,1,2,2,0
610,27,5,1,4,1,1.0,1,3,9,0


In [163]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [164]:
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoded = dept_encoder.transform(y_train[['Department']])
y_test_dept_encoded = dept_encoder.transform(y_test[['Department']])

dept_columns = dept_encoder.get_feature_names_out(['Department'])

df_y_train_dept = pd.DataFrame(y_train_dept_encoded, columns=dept_columns)
df_y_test_dept = pd.DataFrame(y_test_dept_encoded, columns=dept_columns)

y_test_dept_encoded[:5]

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [165]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])

attrition_columns = attrition_encoder.get_feature_names_out(['Attrition'])

df_y_train_attrition = pd.DataFrame(y_train_attrition_encoded, columns=attrition_columns)
df_y_test_attrition = pd.DataFrame(y_test_attrition_encoded, columns=attrition_columns)

y_test_attrition_encoded[:5]

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Part 2: Create, Compile, and Train the Model

In [166]:
# Find the number of columns in the X training data.
num_input_features = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_input_features,), name='input')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)

In [167]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
dept_output = layers.Dense(3, activation='softmax', name='department_output')(dept_hidden)

In [168]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition_output')(attrition_hidden)

In [169]:
# Create the model
model = Model(inputs=input_layer, outputs={'department_output': dept_output,
                                           'attrition_output': attrition_output})

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output' : 'categorical_crossentropy',
                    'attrition_output' : 'categorical_crossentropy'},
              metrics={'department_output' : 'accuracy',
                       'attrition_output' : 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            704 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [170]:
# Train the model
model.fit(X_train_scaled,
          {'department_output' : df_y_train_dept,
           'attrition_output' : df_y_train_attrition},
          epochs=100)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - attrition_output_accuracy: 0.7964 - attrition_output_loss: 0.5267 - department_output_accuracy: 0.5971 - department_output_loss: 0.8998 - loss: 1.4264
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8562 - attrition_output_loss: 0.3776 - department_output_accuracy: 0.6513 - department_output_loss: 0.7884 - loss: 1.1660
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8676 - attrition_output_loss: 0.3663 - department_output_accuracy: 0.6407 - department_output_loss: 0.7630 - loss: 1.1290
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8490 - attrition_output_loss: 0.3574 - department_output_accuracy: 0.6646 - department_output_loss: 0.7520 - loss: 1.1096
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8759 - attrition_output_loss: 0.3385 - department_output_accuracy: 0.6426 - department_output_loss: 0.

In [171]:
# Evaluate the model with the testing data
test_results = model.evaluate(np.array(X_test_scaled),
                              {'department_output' : df_y_test_dept,
                               'attrition_output' : df_y_test_attrition},
                              return_dict=True)

test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8285 - attrition_output_loss: 1.4659 - department_output_accuracy: 0.5015 - department_output_loss: 3.0469 - loss: 4.5179  


{'attrition_output_accuracy': 0.8288043737411499,
 'attrition_output_loss': 1.3462501764297485,
 'department_output_accuracy': 0.5163043737411499,
 'department_output_loss': 3.020684242248535,
 'loss': 4.400179862976074}

In [172]:
# # Print the accuracy for both department and attrition
print(f"Attrition predictions accuracy: {test_results['attrition_output_accuracy']}")
print(f"Department predictions accuracy: {test_results['department_output_accuracy']}")


Attrition predictions accuracy: 0.8288043737411499
Department predictions accuracy: 0.5163043737411499


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is probabaly not the best metric for this data.  Due the imbalance of data in Department, AUC would be a better metric.

2. I chose `softmax` as the activation function for both outputlayers.  Since the labels of Aepartment and Attrition were both one-hot encoded from a single column repectively, they are mutually exclusive.

3. First of all, I would try to add more data due to the imbalance of data in Department.  Secondly, I would try adding layers and units to see if it can capture more complex patterns.  Lastly, I would try to tune the hyperparameters such as batch size, epochs and optimizer of the model.